# Using CleF - Climate Finder to discover ESGF data at NCI

This notebook shows examples of how to use the CleF (Climate Finder) python module to search for ESGF data on the NCI server. <br>
Currently the tool is set up for CMIP5 and CMIP6 data, but other ESGF dataset like CORDEX will be available in the future. <br> 

CleF is installed in the conda module analysis3. This is managed by the CMS and is available simply by running
  >  module use /g/data3/hh5/public/modules <br>
  >  module load conda/analysis3  <br> <br>

You could use the module interactively, for the moment we will use its command line options. <br>
Let's start!

## Command syntax

In [ ]:
! ~/.local/bin/clef

By simpling running the command **clef** with no arguments, the tool shows the help message and then exits, basically it is equivalent to 
> clef --help <br>

We can see currently there are sub-commands, one for each available dataset: **cmip5** and **cmip6**.  <br>
There are also five different options that can be passed before the sub-commands, one we have already seen is *--help*. The others are used to modify how the output of the search. <br>
Lets start from searching some CMIP5 data, to see what we can pass to the **cmip5** sub-command we can simply run it with its *--help* option.

## CMIP5

In [ ]:
! ~/.local/bin/clef cmip5 --help

### Passing arguments and options

The *help* shows all the constraints we can pass to the tool, there are also some additional options which can change the way we run our search. For the moment we can ignore these and use their default values. <br>
Some of the constraints can be passed using an abbreviation,like *-v* instead of *--variable*. This is handy once you are more familiar with the tool. <br>
The same option can have more than one name, for example *--ensemble* can also be passed as *--member*, this is because the terminology has changed between CMIP5 and CMIP6. <br>
You can pass how many constraints you want and pass the same constraint more than once. Let's see what happens though if we do not pass any constraint.

In [ ]:
! ~/.local/bin/clef cmip5

In [ ]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historicals --table day --ensemble r2i1p1

Oops that wasn't reasonable! I mispelled the experiment "historicals" does not exists and the tool is telling me it cannot find any matches.

In [ ]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table days --ensemble r2i1p1

Made another spelling mistake, in this case the tool knows that I passed a wrong value and lists for me all the available options for the CMOR table. Eventually we are aiming to validate all the arguments we can, although for some it is no possible to pass all the possible values (ensemble for example).

In [ ]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table day --ensemble r2i1p1

The tool first search on the ESGF for all the files that match the constraints we passed. It then looks for these file locally and if it finds them it returns their path on raijin.
For all the files it can't find locally, the tool check an NCI table listing the downloads they are working on. Finally it lists missing datasets which are in the download queue, followed by the datasets that are not available locally and no one has yet requested. <br>

The tool list the datasets paths and dataset_ids, if you want you can get a more detailed list by file by passing the *--format file* option. <br>

The search by default returns the latest available version. What if we want to have a look at all the available versions?

In [ ]:
! ~/.local/bin/clef cmip5 --variable tasmin --experiment historical --table Amon -m ACCESS1.0 --all-versions

The option *--all-versions* is the reverse of *--latest*, which is also the default, so we get a list of all available versions. <br>
Since all the ACCESS1.0 data is available on NCI (which is the authoritative source for the ACCESS models) the tool doesn't find any missing datasets and let us know about it.

## CMIP6

In [ ]:
! ~/.local/bin/clef cmip6 --help

The **cmip6** sub-command works in the same way but some constraints are different. As well as changes in terminology CMIP6 has more attributes (*facets*) that can be used to search. <br>
Examples of these are the **activity** which groups experiments, **resolution** which is an approximation of the actual resolution and **grid**.

### Controlling the ouput: clef options

In [ ]:
! ~/.local/bin/clef --local cmip6 -e 1pctCO2 -t Amon -v tasmax -v tasmin -g gr

In this example we used the *--local* option for the main command **clef** to get only the local matching data path as output. <br> 
Note also that:
- we are using abbreviations for the options where available; 
- we are passing the variable *-v* option twice; 
- we used the CMIP6 specific option *-g/--grid* to search for all data that is not on the model native grid. This doesn't indicate a grid common to all the CMIP6 output only to the model itself, the same is true for member_id and other attributes.

In [ ]:
! ~/.local/bin/clef --missing cmip6 -e 1pctCO2 -v clw -v clwvi -t Amon -g gr

This time we used the *--missing* option and the tool returned only the results matching the constraints that are available on the ESGF but not locally (we changed variables to make sure to get some missing data back).

In [ ]:
! ~/.local/bin/clef --remote cmip6 -e 1pctCO2 -v tasmin -v tasmax -t Amon -g gr

The *--remote* option returns the Dataset_ids of the data matching the constraints, regardless that they are available locally or not.

In [ ]:
! ~/.local/bin/clef --remote cmip6 -e 1pctCO2 -v tasmin -v tasmax -t Amon -g gr --format file

Running the same command with the option *--format file* after the sub-command, will return the File_ids instead of the default Dataset_ids. <br>
Please note that *--local*, *--remote* and *--missing* together with *--request*, which we will look at next, are all options of the main command **clef** and they need to come before any sub-commands.

## Requesting new data

What should we do if we found out there's some data we are interested into that has not been downloaded or requested yet? <br>
This is a complex data collection, NCI, in consultation with the community, decided the best way to manage it was to have one point of reference. Part of this agreement is that NCI will download the files and update the database that **clef** is interrrogating. After consultation with the community a priority list was decided and NCI has started downloading anything that falls into it as soon as become available. <br> <br>
Users can then request from the NCI helpdesk, other combinations of variables, experiments etc that do not fall into this list. <br>
The list is available from the NCI climate confluence website: <br>
Even without consulting the list you can use **clef**, as we demonstrated above, to search for the data, if it is not queued or downloaded already **clef** will give you an option to request it from NCI. <br>
Let's see how it works.

In [ ]:
%%bash
~/.local/bin/clef --request cmip6 -e 1pctCO2 -v clw -v clwvi -t Amon -g gr
no

We run the same search which gave us as a result 4 missing datasets but this time we used the *--request* option after **clef**.<br>
The tool will execute the search remotely, then look for matches locally and on the NCI download list. Having find none gives as an option of putting in a request. <br>
It will accept any of the following as a positive answer:
> Y  YES y yes <br>

With anything else or if you don't pass anything it will assume you don't want to put in a request.<br>
It still saved the request in a file we can use later.<br>

In [ ]:
!cat CMIP6_*.txt

If I answered 'yes' the tool would have sent an e-mail to the NCI helpdesk with the text file attached, NCI can pass that file as input to their download tool and queue your request.